# Start Simulation

In [1]:
# env.clean(clean_all=True)

In [2]:
import random
import numpy as np
import os
import time
from PIL import Image

import torch
torch.__version__
device = torch.device("cuda")

In [3]:
import getpass
user = getpass.getuser()
print(user)

yizhou


In [4]:
usd_path = f"omniverse://localhost/Users/{user}/UVA/uva_table.usd"

In [5]:
from omni.isaac.kit import SimulationApp    
simulation_app = SimulationApp({"headless": True, "open_usd": usd_path,  "livesync_usd": usd_path}) 

Starting kit application with the fillowing args:  ['/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.1/exts/omni.isaac.kit/omni/isaac/kit/simulation_app.py', '/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.1/apps/omni.isaac.sim.python.kit', '--/app/tokens/exe-path=/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.1/kit', '--/persistent/app/viewport/displayOptions=3094', '--/rtx/materialDb/syncLoads=True', '--/rtx/hydra/materialSyncLoads=True--/omni.kit.plugin/syncUsdLoads=True', '--/app/renderer/resolution/width=1280', '--/app/renderer/resolution/height=720', '--/app/window/width=1440', '--/app/window/height=900', '--/renderer/multiGpu/enabled=True', '--ext-folder', '/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.1/exts', '--ext-folder', '/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.1/apps', '--/physics/cudaDevice=0', '--portable', '--no-window']
Passing the following args to the base kit application:  ['-f', '/home/yizhou/.local/share/jupyter/runtime/kernel-5857fc03-af4b-

OmniAssetFileFormat


Warp initialized:
   Version: 0.2.2
   CUDA device: NVIDIA GeForce RTX 3090
   Kernel cache: /home/yizhou/.cache/warp/0.2.2
[2.054s] [ext: omni.kit.menu.create-1.0.2] startup
[2.055s] [ext: omni.mdl-0.1.0] startup
[2.076s] [ext: omni.kit.window.file_importer-1.0.4] startup
[2.077s] [ext: omni.kit.window.file_exporter-1.0.4] startup
[2.078s] [ext: omni.kit.material.library-1.3.10] startup
[2.081s] [ext: omni.kit.window.drop_support-1.0.0] startup
[2.081s] [ext: omni.kit.window.file-1.3.16] startup
[2.084s] [ext: omni.kit.context_menu-1.3.9] startup
[2.087s] [ext: omni.kit.window.property-1.6.3] startup
[2.088s] [ext: omni.kit.window.content_browser-2.4.28] startup
[2.100s] [ext: omni.kit.widget.stage-2.6.15] startup
[2.104s] [ext: omni.isaac.version-1.0.0] startup
[2.105s] [ext: omni.kit.property.usd-3.14.9] startup
[2.149s] [ext: omni.kit.viewport.legacy_gizmos-1.0.0] startup
[2.156s] [ext: omni.hydra.rtx-0.1.0] startup
[2.167s] [ext: omni.renderer-rtx-0.0.0] startup
[2.168s] [ext: omn

In [6]:
# set log level
import logging
import carb

logging.getLogger("omni.hydra").setLevel(logging.ERROR)
logging.getLogger("omni.isaac.urdf").setLevel(logging.ERROR)
logging.getLogger("omni.physx.plugin").setLevel(logging.ERROR)

logging.getLogger("omni.isaac.synthetic_utils").setLevel(logging.ERROR)
logging.getLogger("omni.isaac.synthetic_utils.syntheticdata").setLevel(logging.ERROR)
logging.getLogger("omni.hydra.scene_delegate.plugin").setLevel(logging.ERROR)


l = carb.logging.LEVEL_ERROR
carb.settings.get_settings().set("/log/level", l)
carb.settings.get_settings().set("/log/fileLogLevel", l)
carb.settings.get_settings().set("/log/outputStreamLevel", l)

# # This logged error is printed as it should
# carb.log_error("ERROR")
# # This warning is printed but should not
# carb.log_warn("WARNING")

# Config

In [7]:
from task.config import DATA_PATH, FEATURE_PATH
task_type = "Table"
side_choice = "Border"
base_asset_id = 0
load_nucleus = True

In [8]:
pause

NameError: name 'pause' is not defined

# Init Env

In [ ]:
from uva_env import UvaEnv
env = UvaEnv()

In [ ]:
from task.utils import add_scene_default
add_scene_default()

In [ ]:
print(list(env.stage.TraverseAll()))

In [ ]:
env.clean()
env.world.step(render=True)

# Scene

In [ ]:
from task.scene import ArrangeScene
scene = ArrangeScene(task_type, side_choice, base_asset_id = 0, traj_id = 0, load_nucleus = load_nucleus)
env.scene = scene

In [ ]:
# add base
scene.add_base_asset()

In [ ]:
# add room
scene.add_room()

In [ ]:
env.world.step(render=True)

In [ ]:
# randomize scene
from layout.randomizer import Randomizer
randomizer = Randomizer(load_nucleus = load_nucleus)

# Reward

In [ ]:
from uv.reward import Rewarder

In [ ]:
rewarder = Rewarder(env.world)
env.rewarder = rewarder

# Render

In [ ]:
from render.helper import RenderHelper
render = RenderHelper(task_type, side_choice)

render.add_task_cameras(camera_type="main")
render.add_task_cameras(camera_type="side")
render.set_cameras()

In [ ]:
render.camera_paths

In [ ]:
pause

#  Learning

In [ ]:
# from learning.network.resnet import ResNetFeatureExtractor

# feature extraction
from learning.utils import extract_image_clip_feature_and_save, obtain_action_from_trainer
from transformers import CLIPProcessor, CLIPModel

feature_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
feature_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# replay buffer
import json
from learning.replay_buffer import ReplayBuffer
from learning.config import *

buffer = ReplayBuffer(max_size=2000)

# trainer
from learning.network.sac import *

policy = Policy()

qf1 = QFunction()
qf2 = QFunction()
target_qf1 = QFunction()
target_qf2 = QFunction()

from learning.sac_trainer import SACTrainer
from learning.ddpg_trainer import ddpg_trainer

trainer = SACTrainer(policy, qf1, qf2, target_qf1, target_qf2, 
     use_automatic_entropy_tuning = False, 
     policy_lr=1e-3, 
     qf_lr=1e-3,
     target_update_period = 5)

# Trajectory

In [ ]:
use_network = True
debug = True

In [ ]:
total_traj = 0
total_step = 0

---------------------

In [ ]:
# traj config
for traj_id in range(1):
    print("total_traj: ", total_traj)
    env.scene.traj_id = traj_id
    
    # base
    if total_traj % 5 == 0:
        asset_count = len(env.scene.base_asset_file_paths)
        base_asset_id = random.choice([_ for _ in range(asset_count)])
        env.scene.base_asset_id = base_asset_id    
        env.clean(clean_base = True, clean_all = False)
        scene.add_base_asset()
        
        # randomize scene
        randomizer.randomize_house(rand = True)
        
    image_folder = os.path.join(DATA_PATH, task_type, side_choice, str(traj_id))
    
    env.world.step(render = True)
    env.world.step(render = True)

    # get images
    env.world.render()
    images = render.get_images()
    render.save_rgb(images[0]['rgb'], image_folder, "0")
    render.save_rgb(images[1]['rgb'], image_folder, "0_side")
    
    ## extract feature
    
#     extract_image_feature_and_save(images[0]['rgb'][:,:,:3], 
#         feature_extractor, os.path.join(image_folder, str(0) + ".pt"))
    if use_network:
        extract_image_clip_feature_and_save(f"{image_folder}/{0}.png", feature_model, feature_processor, 
            f"{image_folder}/{0}.pt",)
    

    # trajectory
    for step in range(5):
        total_step += 1
        
        # sample an object
        env.add_scene_obj(mode = "random")
        
        # TODO: get action from sampling
        if not use_network or total_traj < 10 or np.random.rand() < 0.2:
            x, y = np.tanh(np.random.randn()), np.tanh(np.random.randn())
        else:
            image_feature_file = f"{image_folder}/{step}.pt"
            
            object_info = env.scene.objects[-1]
            object_type = object_info["type"]
            obj_name = object_info["name"][:-4]
            object_feature_file = os.path.join(FEATURE_PATH, object_type, obj_name + ".pt")
            x, y = obtain_action_from_trainer(image_feature_file, object_feature_file, trainer, 
                                              scaler=np.exp(- total_traj / 100))
            object_info["use_network"] = True
        
        # load the object into the scene
        env.put_last_object((x, y)) 
        env.world.step(render=True)
        
        # register the object to the world for physics update
        env.register_last_object()
        env.world.step(render=True)

        # get images
        env.world.render()
        images = render.get_images()
        render.save_rgb(images[0]['rgb'], image_folder, str(step + 1))
        render.save_rgb(images[1]['rgb'], image_folder, f"{step + 1}_side")

        ## calculate reward
        env.calculate_last_reward(simulation_step = 30)
        
        ## extract feature
#         extract_image_feature_and_save(images[0]['rgb'][:,:,:3], 
#             feature_extractor, os.path.join(image_folder, str(step + 1) + ".pt"))
        if use_network:
            extract_image_clip_feature_and_save(f"{image_folder}/{step + 1}.png", feature_model, feature_processor, 
                f"{image_folder}/{step + 1}.pt",)
    

        ## reset
        env.world.reset()
        env.world.step(render=True)
        
        ## trainer nework
        if use_network and total_step % UPDATE_TRAINER_STEPS == 0 and total_traj > 5:
            batch = buffer.sample_batch(batch_size = BATCH_SIZE)
            trainer.update(batch)
            
            if debug and total_step % 10 == 0:
                rewards = batch['rewards'].to(device)
                terminals = batch['terminals'].to(device)
                obs = batch['observations'].to(device)
                actions = batch['actions'].to(device)
                next_obs = batch['next_observations'].to(device)
                obj_features = batch['object_features'].to(device)
                
                dist = trainer.policy(obs, obj_features)
                pred = trainer.qf1(obs, obj_features, actions)
                print(f"debug {total_traj}/{total_step}", #"\n dist: ", dist.mean.flatten().tolist(), dist.stddev.flatten().tolist(),
                      "\n pred:", pred.flatten().tolist(),
                      "\n rewards: ", rewards.flatten().tolist())


    # Record
    record = env.scene.get_scene_data()
    env.scene.save_scene_data()
    # print("record: ", record)
    
    # Add record to buffer
    if use_network:
        buffer.add_scene_sample(record)

    # Reset (env clean)
    env.clean(clean_all = False)
    env.step(render = True)
    total_traj += 1

# Debug

In [ ]:
env.clean(clean_base = True, clean_all = False)

In [ ]:
# get images
env.world.render()
env.world.render()
images = render.get_images()

Image.fromarray(images[0]['rgb'], "RGBA").resize((300, 300))

##  Debug Training

In [9]:
import json
from learning.replay_buffer import ReplayBuffer

buffer = ReplayBuffer(max_size=1000)

for i in range(1):
    replay = json.load(open(f"{DATA_PATH}/{task_type}/{side_choice}/{i}/scene.json"))
    buffer.add_scene_sample(replay)

In [10]:
len(buffer.dataset)

5

In [11]:
model_type = "ddpg"

In [12]:
if model_type == "sac":
    # trainer
    from learning.network.sac import *

    policy = Policy()
    target_policy = Policy()

    qf1 = QFunction()
    qf2 = QFunction()

    target_qf1 = QFunction()
    target_qf1.eval()

    target_qf2 = QFunction()
    target_qf2.eval()

    from learning.sac_trainer import SACTrainer

    trainer = SACTrainer(policy, qf1, qf2, target_qf1, target_qf2, 
         use_automatic_entropy_tuning = True, 
         policy_lr=1e-3, 
         qf_lr=1e-3,
         target_update_period = 5)

else: # ddpg
    # trainer
    from learning.network.ddpg import *

    policy = Policy()
    target_policy = Policy()

    qf1 = QFunction()
    target_qf1 = QFunction()
    target_qf1.eval()

    from learning.ddpg_trainer import DDPGTrainer
    
    trainer = DDPGTrainer(qf1, target_qf1, policy, target_policy,
     policy_learning_rate=1e-3,
     qf_learning_rate=1e-3
    )

In [13]:
self = trainer

In [14]:
self.policy(obs, obj_features)

NameError: name 'obs' is not defined

In [21]:
for _ in range(100):
    batch = buffer.sample_batch(batch_size=4)

    rewards = batch['rewards'].to(device)
    terminals = batch['terminals'].to(device)
    obs = batch['observations'].to(device)
    actions = batch['actions'].to(device)
    next_obs = batch['next_observations'].to(device)

    obj_features = batch['object_features'].to(device)

    # print("rewards", rewards)

    self.update(batch)
    
    if _ % 10 == 0:
        dist = self.policy(obs, obj_features)
        pred = self.qf(obs, obj_features, actions)
        print(_, "\n mean   : ", [round(x, 3) for x in dist.flatten().tolist()], 
              "\n pred   :", [round(x, 3) for x in pred.flatten().tolist()],
              "\n rewards: ", [round(x, 3) for x in rewards.flatten().tolist()])
    
#     # debug
#     pred = self.qf1(obs, obj_features, actions)
#     loss = self.qf_criterion(pred, rewards)
    
#     self.qf1_optimizer.zero_grad()
#     loss.backward()
#     self.qf1_optimizer.step()
    
#     if _ % 20 == 0:
#         print(_, "\n debug loss: ", loss.item(), "\n pred:", 
#               pred.flatten().tolist(), "\n rewards: ", rewards.flatten().tolist())

0 
 mean   :  [0.221, 0.825, 0.265, 0.786, 0.25, 0.799, 0.3, 0.696] 
 pred   : [-0.766, -0.734, 0.261, 0.942] 
 rewards:  [-0.431, -0.431, 0.301, 0.959]
10 
 mean   :  [0.009, 0.997, 0.002, 1.0, 0.001, 1.0, 0.002, 1.0] 
 pred   : [0.68, -0.472, -0.453, 0.164] 
 rewards:  [0.959, -0.431, -0.431, 0.301]
20 
 mean   :  [0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0] 
 pred   : [-0.455, 0.265, -0.432, 0.354] 
 rewards:  [-0.431, 0.301, -0.431, 0.373]
30 
 mean   :  [0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0] 
 pred   : [0.475, 0.673, 1.311, -0.158] 
 rewards:  [0.301, 0.373, 0.959, -0.431]
40 
 mean   :  [0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0] 
 pred   : [-0.147, 1.515, 0.645, 0.839] 
 rewards:  [-0.431, 0.959, 0.301, 0.373]
50 
 mean   :  [0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0] 
 pred   : [1.838, 0.053, 0.352, 1.117] 
 rewards:  [0.959, -0.431, -0.431, 0.373]
60 
 mean   :  [0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0] 
 pred   : [1.056, 2.133, 0.589, 0.281] 
 rewards:  [0.301, 0.959, -0.431, -0.431]


In [ ]:
dist = self.policy(obs, obj_features)

In [ ]:
dist = self.policy(obs.to(device), obj_features.to(device))

In [ ]:
dist.normal_mean, dist.stddev

In [ ]:
dist.sample(scaler = 0.2)

In [ ]:
self = dist

In [ ]:
scaler = 0.1

In [ ]:
self.normal_mean + scaler * self.normal_std * MultivariateDiagonalNormal(
    torch.zeros(self.normal_mean.size()).to(self.device),
    torch.ones(self.normal_std.size()).to(self.device)
).sample()

# distribution

In [ ]:
from learning.distributions import MultivariateDiagonalNormal

In [ ]:
dist = JustNormal(dist.mean, dist.stddev)

In [ ]:
dist.rsample_and_logprob()